In [1]:
pip install pyspark

     |████████████████████████████████| 281.3 MB 36 kB/s 
     |████████████████████████████████| 198 kB 13.7 MB/s 
  Created wheel for pyspark: filename=pyspark-3.2.0-py2.py3-none-any.whl size=281805912 sha256=3d3cc6cf1bc337e52846ec46e6e666fc88630e010f10bbdd6149c0baba5ca1de
  Stored in directory: /root/.cache/pip/wheels/0b/de/d2/9be5d59d7331c6c2a7c1b6d1a4f463ce107332b1ecd4e80718
Successfully built pyspark


In [2]:
from pyspark.sql import SparkSession
import pyspark.sql.functions as F

In [3]:
spark = (
    SparkSession.builder
      .master('local')
      .appName('dataframe_withcolumn')
      .config('spark.ui.port', '4050')
      .getOrCreate()
)

In [4]:
df1 = (spark
       .read
       .format("csv")
       .option("header", "true")
       .option("inferschema", "true")
       .option("delimiter", ";")
       .load('/content/drive/MyDrive/Colab Notebooks/arquivo_geral.csv')
)

WHEN - OTHERWISE


In [ ]:
'''CRIAR UMA NOVA COLUNA COM AS SEGUINTES CONDIÇÕES: 
   CASO O NÚMERO DE CASOS NOVOS SEJA MAIOR DO QUE ZERO, COLOCAR O TEXTO TEM CASOS NOVOS
   CASO O NÚMERO DE CASOS NOVOS NÃO SEJA MAIOR DO QUE ZERO, COLOCAR O TEXTO SEM CASOS NOVOS
'''
df2 = (df1.withColumn("status", F.when(F.col("casosNovos") > 0, "Tem casos novos")
          .otherwise("Sem casos novos"))    
      )

df2.select(F.col("regiao"), F.col("estado"), F.col("casosNovos"), F.col("status")).show(500)

PASSANDO A QUANTIDADE DE CASOS NOVOS E CONCATENANDO COM O TEXTO 'CASOS NOVOS'


In [15]:
df2.show(5)


+------+------+----------+----------+---------------+-----------+----------------+---------------+
|regiao|estado|      data|casosNovos|casosAcumulados|obitosNovos|obitosAcumulados|         status|
+------+------+----------+----------+---------------+-----------+----------------+---------------+
| Norte|    RO|2020-01-30|         0|              0|          0|               0|Sem casos novos|
| Norte|    RO|2020-01-31|         0|              0|          0|               0|Sem casos novos|
| Norte|    RO|2020-02-01|         0|              0|          0|               0|Sem casos novos|
| Norte|    RO|2020-02-02|         0|              0|          0|               0|Sem casos novos|
| Norte|    RO|2020-02-03|         0|              0|          0|               0|Sem casos novos|
+------+------+----------+----------+---------------+-----------+----------------+---------------+
only showing top 5 rows



In [ ]:
df3 = (df1.withColumn("status", 
          F.when(F.col("casosNovos") > 0, F.concat(df1.casosNovos,F.lit(" casos novos")))
          .otherwise("Não tem casos novos")           
                     )
      )
df3.select(F.col("data"),F.col("casosNovos"),F.col("status") ).show(250)

In [ ]:
'''
Criando colunas com Dia/Mês/ANO para ficar mais organizado no DataFrame
Isso é feito com a utilização da withColumn que cria uma nova coluna junto com a substring que mostra uma
parte da string de acordo com os parâmetros escolhidos
'''

F.substring(texto, 1, 5)
df1.select("data").show(5)

In [ ]:
dfdia = df3.withColumn("dia", F.lit(F.substring('data', 9, 2)))
dfdia.show(50)

In [ ]:
dfmes = df3.withColumn("mes", F.lit(F.substring('data', 6, 2)))
dfmes.show(50)

In [ ]:
dfano = df3.withColumn("ano", F.lit(F.substring('data', 1, 4)))
dfano.show(50)

In [ ]:
dfdata = df3.withColumn("dia", F.lit(F.substring('data', 9, 2)))
dfdata = dfdata.withColumn("mes", F.lit(F.substring('data', 6, 2)))
dfdata = dfdata.withColumn("ano", F.lit(F.substring('data', 1, 4)))
dfdata.show(10)

Tudo junto

In [23]:
df4 = (
    df1.withColumn("Status",
                   F.when(F.col("casosNovos") > 0, 
                          F.concat(df1.casosNovos, F.lit(" caso(s) Novo(s)")))
                   .otherwise("Sem Casos Novos")
                   )
        .withColumn("Dia", F.substring(F.col("data"), 9, 2))
        .withColumn("Mes", F.substring(F.col("data"), 6, 2))
        .withColumn("Ano", F.substring(F.col("data"), 1, 4))
)
df4.show()

+------+------+----------+----------+---------------+-----------+----------------+---------------+---+---+----+
|regiao|estado|      data|casosNovos|casosAcumulados|obitosNovos|obitosAcumulados|         Status|Dia|Mes| Ano|
+------+------+----------+----------+---------------+-----------+----------------+---------------+---+---+----+
| Norte|    RO|2020-01-30|         0|              0|          0|               0|Sem Casos Novos| 30| 01|2020|
| Norte|    RO|2020-01-31|         0|              0|          0|               0|Sem Casos Novos| 31| 01|2020|
| Norte|    RO|2020-02-01|         0|              0|          0|               0|Sem Casos Novos| 01| 02|2020|
| Norte|    RO|2020-02-02|         0|              0|          0|               0|Sem Casos Novos| 02| 02|2020|
| Norte|    RO|2020-02-03|         0|              0|          0|               0|Sem Casos Novos| 03| 02|2020|
| Norte|    RO|2020-02-04|         0|              0|          0|               0|Sem Casos Novos| 04| 0

In [ ]:
df4 = (
    df1.withColumn("Dia", F.substring(F.col("data"), 9, 2).cast("integer"))
        .withColumn("MES", F.substring(F.col("data"), 6, 2).cast("integer"))
        .withColumn("ANO", F.substring(F.col("data"), 1, 4).cast("integer"))
)
lista_esquema = ["regiao", "data", "dia", "mes", "ano"]
df4.select(lista_esquema).show(10)